# Mexican federal budget pre-processing pipeline

## Instructions

To you run the notebook:

1. choose a unique `ITERATION_LABEL` for each pipeline run
2. specify and describe your input files (`INPUT_FILES`)
3. make sure your column mapping (`COLUMN_ALIASES`) is correct
3. run the whole notebook by clicking on __Kernel > Restart & Run All__

## Settings

Choose a unique iteration label for each pipeline run.

In [1]:
ITERATION_LABEL = 'iteration-10-development'

Put your input files inside the `pipeline.in` folder and describe them here.

In [2]:
INPUT_FILES = {
#     2010: {'name': 'Cuenta_Publica_2010.csv', 'encoding': 'windows-1252'},
#     2011: {'name': 'Cuenta_Publica_2011.csv', 'encoding': 'windows-1252'},
#     2012: {'name': 'Cuenta_Publica_2012.csv', 'encoding': 'windows-1252'},
#     2013: {'name': 'Cuenta_Publica_2013.csv', 'encoding': 'windows-1252'},
#     2014: {'name': 'Cuenta_Publica_2014.csv', 'encoding': 'windows-1252'},
#     2015: {'name': 'Cuenta_Publica_2015.csv', 'encoding': 'windows-1252'},
#     2016: {'name': '2016_2T_Gasto_OS.csv', 'encoding': 'windows-1252'} # cp850 for the original 2016 file
    2013: {'name': 'CP_2013.csv', 'encoding': 'cp850'}
}

If your input files don't all have the same column names, define your mapping here. 

In [3]:
COLUMN_ALIASES = {
    'Actividad Institucional': ['AI'],
    'Adefas': ['ADEFAS'],
    'Aprobado': [
        'PEF_2016',
        'Importe Presupuesto de Egresos de la Federación',
        'Importe Presupuesto de Egresos de la Federación (PEF)'
    ],
    'Ciclo': None,
    'Clave de cartera': ['CLAVE_CARTERA'],
    'Descripción de Fuente de Financiamiento': ['FUENTE_FINAN_DESCRIPCION'],
    'Descripción de Función': ['FUNCIONL_DESCRIPCION'],
    'Descripción de Grupo Funcional': [
        'Descripción de Finalidad',
        'GRUPO_FUN_DESCRIPCION',
        'Descripción de Grupo Funcional'
    ],
    'Descripción de Objeto del Gasto': ['CONCEPTO_DESCRIPCION'],
    'Descripción de Programa Presupuestario': ['PROGR_PRES_DESCRIPCION'],
    'Descripción de Ramo': ['RAMO_DESCRIPCION'],
    'Descripción de Reasignacion': ['REASIGNACION_DESCRIPCION'],
    'Descripción de Subfunción': ['SUBFUNCIONL_DESCRIPCION', 'Descripción de subfunción'],
    'Descripción de Tipo de Gasto': ['TIPO_GASTO_DESCRIPCION'],
    'Descripción de Unidad Responsable': ['UNIDAD_DESCRIPCION'],
    'Descripción de la Actividad Institucional': [
        'ACTIVIDAD_INST_DESCRIPCION',
        'Descripción de Actividad Institucional'
    ],
    'Descripción de Entidad Federativa': ['Descripción de la entidad federativa', 'ENTIDAD_FED_DESCRIPCION'],
    'Descripción de la modalidad del programa presupuestario': [
        'MODALIDAD_DESCRIPCION',
        'Descripción del Identificador del Programa Presupuestario',
        'Descripción del Identificador de Programa Presupuestario'
    ],
    'Devengado': None,
    'Ejercicio': None,
    'Ejercido': None,
    'Entidad Federativa': ['EF'],
    'Fuente de Financiamiento': ['FF', 'Fuente de Finaciamiento'],
    'Función': ['FN'],
    'Grupo Funcional': [
        'Finalidad', 'GF', 'Grupo Funcional'
    ],
    'Modalidad del Programa presupuestario': [
        'MOD',
        'Identificador de Programa Presupuestario',
        'Identificador del Programa Presupuestario'
    ],
    'Modificado': None,
    'Objeto del Gasto': ['CONCEPTO'],
    'Pagado': None,
    'Programa Presupuestario': ['PP'],
    'Ramo': ['RAMO'],
    'Reasignacion': ['RA'],
    'Subfunción': ['SF'],
    'Tipo de Gasto': ['TG'],
    'Unidad Responsable': ['UNIDAD'],
    'Capitulo': None,
    'Concepto': None,
    'Partida Genérica': None,
    'Partida Específica': None,
    'Descripción de Capitulo': None,
    'Descripción de Concepto': None,
    'Descripción de Partida Genérica': None,
    'Descripción de Partida Específica': ['Descripcion de Partida Específica'],    
}

The following hierarchical categories will have IDs prefixed with the parent categories:

In [4]:
HIERARCHIES = {
    'functional': [
        'Grupo Funcional', 
        'Función', 
        'Subfunción', 
        'Actividad Institucional'
    ],
    'administrative': [
        'Ramo', 
        'Unidad Responsable'
    ],
    'activities': [
        'Modalidad del Programa presupuestario', 
        'Programa Presupuestario'
    ],
}

The following columns are unsused and removed at the end of the pipeline:

In [5]:
REMOVE_OUTPUT_COLUMNS = [
    'Reasignacion',
    'Objeto del Gasto',
    'Descripción de Reasignacion',
    'Descripción de Objeto del Gasto'
]

In [6]:
REMOVE_INPUT_COLUMNS = {
    2016: [
        'Adefas',
        'Partida Específica',
        'Partida Genérica',
        'Descripción de Partida Genérica',
        'Descripcion de Partida Específica',
        'Ejercicio',
        'Devengado',
        'Ejercido',
    ]
}

That's it. Now just run the notebook from beginning to end.

## Imports

In [7]:
from sys import stdout
from pandas import read_csv, concat, DataFrame, ExcelWriter, ExcelFile, Series
from numpy import nan, isnan
from os.path import join, isdir
from os import mkdir
from json import dumps, loads
from pprint import pprint

## Configuration

In [8]:
BASENAME = 'mexican_federal_budget'
INPUT_FOLDER = 'pipeline.in'
OUTPUT_FOLDER = 'pipeline.out'
ITERATION_FOLDER = join(OUTPUT_FOLDER, ITERATION_LABEL)
MERGED_FILE = join(ITERATION_FOLDER, BASENAME + '.merged.csv')
CATALOGS_FOLDER = 'objeto_del_gasto.catalog'
CATALOGS_FILE = 'objeto_del_gasto.catalog.xlsx'

In [9]:
if isdir(ITERATION_FOLDER):
    raise ValueError('Please enter a unique iteration label')
    
mkdir(ITERATION_FOLDER)

## Encoding inspection

Detect the file encodings of the input files using the `cChardet` utility library. __Warning:__ it's not always accurate. This is meant only as an indication only. In the end, encodings will be taken from `INPUT_FILES`.

In [10]:
def detect_encodings():
    """Detect CSV file encoding with the cChardet library"""

    try:
        import cchardet as chardet
    except ImportError:
        cChardet = 'https://github.com/PyYoshi/cChardet'
        print('Encoding inspection skipped: install %s', cChardet)
        return

    results = {}
    results_file = join(OUTPUT_FOLDER, ITERATION_LABEL, 'encodings.detected.json')
    
    for year, file in sorted(INPUT_FILES.items()):
        datafile = join(INPUT_FOLDER, file['name'])
        
        with open(datafile, 'rb') as f:
            text = f.read()
            
        result = chardet.detect(text)
        results.update({year: result})
        print(year, 'Inspected', file['name'], result)
    
    with open(results_file, 'w+') as json:
        json.write(dumps(results, indent=4))
        print('\nSaved encoding detection report to', results_file)
        
# detect_encodings()

## Load files

In [11]:
def read_columns(file, encoding):
    """Return clean CSV file headers"""
    
    with open(file, encoding=encoding) as csv:
        header = csv.readline()
        return header.replace('\n', '').split(',')

In [12]:
def force_strings(columns):
    """Return string enforcement for each column of a CSV file"""
    
    for column in columns:
        yield column, str

In [13]:
def load_csv_files():
    """Load raw data (CSV) files"""
    
    batch = {}
    
    for year, file in sorted(INPUT_FILES.items()):
        filepath = join(INPUT_FOLDER, file['name'])
        column_names = read_columns(filepath, file['encoding'])
        column_types = dict(force_strings(column_names))
        
        batch[year] = read_csv(filepath, encoding=file['encoding'], dtype=column_types)
        print('Loaded', file['name'], 'with encoding', file['encoding'])
    
    print()
    stdout.flush()

    for year in sorted(INPUT_FILES.keys()):
        if year in REMOVE_INPUT_COLUMNS:
            for column in REMOVE_INPUT_COLUMNS[year]:
                try:
                    del batch[year][column]
                    print(year, 'deleted', column)
                except KeyError:
                    print(year, column, 'not found in', file['name'])

        stdout.flush()

    return batch

## Clean the data

In [14]:
def strip_cell_padding(batch):
    for year in sorted(batch.keys()):
        for column in batch[year].columns:
            batch[year].rename(columns={column: column.strip()}, inplace=True)
            batch[year][column] = batch[year][column].apply(lambda x: x.strip() if x is not nan else x)
        print(year, 'stripped cell paddings')
        stdout.flush()

In [15]:
def delete_empty_columns(batch):
    for year in batch.keys():
        for column in batch[year].columns:
            if 'Unnamed:' in column:
                try:
                    del batch[year][column]
                    print(year, column, 'deleted')
                    stdout.flush()
                except KeyError:
                    pass  

In [16]:
def count_missing_values(batch):
    collector = {}
    table = []

    for column in get_union_of_columns(batch):
        row = {'Column': column}
        collector.update({column: []})
        
        for year in batch.keys():
            if column in batch[year].columns:
                is_empty = batch[year][column].isnull()
                empty_lines = batch[year].where(is_empty).dropna(how='all')
                collector[column].extend(empty_lines.to_dict(orient='records'))
                nb_empty_cells = len(empty_lines)
            else:
                nb_empty_cells = nan
                
            row.update({year: nb_empty_cells})
            if nb_empty_cells not in (nan, 0):
                print(year, 'found', nb_empty_cells, 'missing values in', column)

        table.append(row)
        
    ordered_columns = ['Column']
    ordered_columns.extend(sorted(batch.keys()))
    empty_values_overview_table = DataFrame(table).reindex_axis(ordered_columns, axis=1)
    
    return empty_values_overview_table, collector

In [17]:
def count_duplicates(batch):
    for year, df in sorted(batch.items()):
        nb_duplicate_lines = df.duplicated().apply(lambda x: 1 if x is True else 0).sum()
        print(year, 'found', nb_duplicate_lines, 'duplicate lines')

## Alias column names

In [18]:
def get_union_of_columns(batch):
    union = set()
    for year in batch.keys():
        union = union | set(batch[year].columns)
    return union

In [19]:
from yaml import load

def load_aliases(file):
    with open(file) as yaml:
        aliases = load(yaml.read())
        return aliases

In [20]:
def map_columns_to_aliases(batch, list_of_aliases):
    for year in sorted(batch.keys()):
        for column in sorted(batch[year].columns):
            if not column in list_of_aliases:
                for reference, aliases in list_of_aliases.items():
                    if aliases:
                        if column in aliases:
                            batch[year].rename(columns={column: reference}, inplace=True)
                            print(year, column, 'replaced with', reference)
                            stdout.flush()
                            break  
                else:
                    print(year, 'NO ALIAS REGISTERED FOR', column)
                    stdout.flush()

In [21]:
def build_overview(batch):
    table = []
    
    for column in get_union_of_columns(batch):
        row = {'Column': column}
        for year in batch.keys():
            row.update({year: column in batch[year].columns})
        table.append(row)
        
    ordered_columns = ['Column']
    ordered_columns.extend(sorted(batch.keys()))
    
    overview = DataFrame(table).reindex_axis(ordered_columns, axis=1)
    print('Column mapping overview: done')
    return overview

## Check expenditure sums

There's a little cleaning to do on the amount columns (zeros represented by a dash). Assume thousands are seperated by a comma.

In [22]:
EXPENDITURE_COLUMNS = [
    'Ejercido', 
    'Devengado', 
    'Aprobado', 
    'Pagado', 
    'Modificado', 
    'Adefas', 
    'Ejercicio'
]
count = 0

def clean_expenditure_columns(batch):
    check_sums = []

    for column in EXPENDITURE_COLUMNS:
        row = {'Column': column}
        
        for year in sorted(batch.keys()):
            try:
                series = batch[year][column]
                
                # I'm assuming a single '-' represents zero
                series = series.apply(lambda x: '0' if x == '-' else x)
                try:
                    series = series.apply(lambda x: x.replace(',', '') if x is not nan else x)    
                except AttributeError:
                    if count < 10:
                        print(year, column)
                batch[year][column] = series.astype(float)
                check_sum = batch[year][column].sum()
                
                print(year, 'cleaned and summed', column, '=', check_sum, 'pesos')
                
            except KeyError:
                check_sum = nan
                
            row.update({year: check_sum})
        
        check_sums.append(row)

    ordered_columns = ['Column']
    ordered_columns.extend(sorted(batch.keys()))
    return DataFrame(check_sums).reindex_axis(ordered_columns, axis=1)    

## Objeto del Gasto Column split

In [23]:
from os.path import join

def generate_catalog(file):
    
    catalog_ = {}
    catalog_file = ExcelFile(file)
    INDEX_COLUMN = 0
    
    for sheet in catalog_file.sheet_names:
        if sheet != 'Concatenated':
            name = sheet.lower().replace(' ', '_')
            output = join('objeto_del_gasto.catalog', name + '.csv')

            df = catalog_file.parse(sheet).dropna()
            index = df.columns[INDEX_COLUMN]

            df[index] =  df[index].astype(str)
            df.set_index(index, inplace=True)
            df = df.groupby(df.index).first()
            df.sort_index(inplace=True)
            
            message = 'Loaded catalog {sheet} into "{name}" ({nb} lines)'
            parameters = dict(sheet=sheet, name=name, nb=len(df))

            print(message.format(**parameters))
            catalog_[name] = df['DESCRIPCION']
    
    print()
    return catalog_

__Note!__ Years are hard coded in the script below.

In [24]:
def split_objeto_del_gasto(batch):
    catalog = generate_catalog(CATALOGS_FILE)
    missing_in_catalog = []
    
    def has_digits(n, N):
        return not isinstance(n, float) and len(n) >= N 
            

    def lookup(n, table, year):
        try:
            return catalog[table].loc[n]
        except KeyError:
            missing_in_catalog.append({'year': year, 'table': table, 'ID': n})
            return nan
        except TypeError:
            # n is nan
            return nan
    
    for year in sorted(batch.keys()):
        if year == 2016:
            print('Skipping', year, 'because the raw CSV already has the required columns')
        
        else:
            objeto = batch[year]['Objeto del Gasto'].astype(str)

            batch[year]['Capitulo'] = objeto.apply(lambda x: x[0] + '000' if x not in (nan, 'nan') else nan)
            batch[year]['Concepto'] = objeto.apply(lambda x: x[:2] + '00' if x not in (nan, 'nan') else nan)
            batch[year]['Descripción de Capitulo'] = batch[year]['Capitulo'].map(lambda x: lookup(x, 'capitulo', year))  
            batch[year]['Descripción de Concepto'] = batch[year]['Concepto'].map(lambda x: lookup(x, 'concepto', year))  
            
            nb_generica_digits = 4 if year in (2008, 2009, 2010) else 3
            
            # Skip the LAST year of the dataset (currently 2016) it has split columns already
            batch[year]['Partida Genérica'] = objeto.apply(lambda x: x[:nb_generica_digits] if has_digits(x, 4) else nan)
            batch[year]['Descripción de Partida Genérica'] = batch[year]['Partida Genérica'].map(lambda x: lookup(x, 'partida_generica', year))  
            
            if year not in (2008, 2009, 2010):
                batch[year]['Partida Específica'] = objeto.apply(lambda x: x if has_digits(x, 5) else nan)
                batch[year]['Descripción de Partida Específica'] = batch[year]['Partida Específica'].map(lambda x: lookup(x, 'partida_específica', year) if has_digits(x, 5) else nan)  
            else:
                batch[year]['Partida Específica'] = nan
                batch[year]['Descripción de Partida Específica'] = nan

            print(year, 'broke down "Objeto del Gasto" column')
        
    return DataFrame(missing_in_catalog).drop_duplicates(['ID', 'table'])

## Prefix IDs 
Disambiguating sub-categories may require prefixing their IDs with their parents' IDs.

In [25]:
def prefix_ids(batch):
    for year in batch.keys():       
        for hierarchy, levels in HIERARCHIES.items():
            prefix = batch[year]['Ciclo'].apply(lambda x: '')
            for n, level in enumerate(levels):
                dash = '.' if n > 0 else ''
                prefix = prefix + dash + batch[year][level]  
                batch[year][level] = prefix
                
                print(year, 'prefixed', hierarchy, 'level', n, level)
                stdout.flush()

## Remove unused columns

In [26]:
def remove_unused_columns(batch):
    for year, budget in batch.items():
        for column in REMOVE_OUTPUT_COLUMNS:
            try:
                del budget[column]
                print(year, 'deleted', column)
            except KeyError:
                pass

##  Pipeline

In [27]:
def do_pipeline():

    def echo_section(section):
        print('\n', section, '\n')

    echo_section('Loading files')
    datasets = load_csv_files()
    
    echo_section('Delete empty columns')
    delete_empty_columns(datasets)

    echo_section('Stripping padding from cells')
    strip_cell_padding(datasets)
    
    echo_section('Counting duplicate lines (NOT de-duplicating)')
    count_duplicates(datasets)
    
    echo_section('Mapping column to aliases')
    map_columns_to_aliases(datasets, COLUMN_ALIASES)

    echo_section('Counting missing values')
    missing_values_report, bad_records = count_missing_values(datasets)
    
    echo_section('Building column mapping overview')
    column_mapping_report = build_overview(datasets)
    
    echo_section('Cleaning expenditure columns')
    sums_report = clean_expenditure_columns(datasets)
    
    echo_section('Breaking down Objeto del Gasto column')
    missing_catalog_ids = split_objeto_del_gasto(datasets)
        
    echo_section('Prefixing IDs of certain category hierarchies')
    prefix_ids(datasets)

    echo_section('Removing unused columns')
    remove_unused_columns(datasets)

    echo_section('Saving pipeline configuration')

    reports_file = join(ITERATION_FOLDER, BASENAME + '.reports.xlsx')
    writer = ExcelWriter(reports_file)    
    missing_values_report.to_excel(writer, 'missing values', encoding='utf-8', index=False)
    column_mapping_report.to_excel(writer, 'column mapping', encoding='utf-8', index=False)
    sums_report.to_excel(writer, 'check sums', encoding='utf-8', index=False)
    missing_catalog_ids.to_excel(writer, 'missing_catalog_IDs', encoding='utf-8', index=False)    
    print('Saved 4 reports to', reports_file)    

    aliases_file = join(ITERATION_FOLDER, BASENAME + '.aliases.json')
    inputs_file = join(ITERATION_FOLDER, BASENAME + '.inputs.json')
    levels_file = join(ITERATION_FOLDER, BASENAME + '.levels.json')
    bad_records_file = join(ITERATION_FOLDER, BASENAME + '.missing.json')

    with open(bad_records_file, 'w+') as json:
        json.write(dumps(bad_records, indent=4))
        
    with open(aliases_file, 'w+') as json:
        json.write(dumps(COLUMN_ALIASES, indent=4))
        
    with open(levels_file, 'w+') as json:
        json.write(dumps(HIERARCHIES, indent=4))
        
    with open(inputs_file, 'w+') as json:
        json.write(dumps(INPUT_FILES, indent=4))
    
    print('Saved input configuration to', inputs_file)    
    print('Saved column aliases to', aliases_file) 
    print('Saved bad records (those with empty cells) to', bad_records_file)    
    print('Saved hierarchy levels used for prefixing to', levels_file) 
    
    echo_section('Pipeline run "%s" done' % ITERATION_LABEL)

    return datasets, missing_catalog_ids, column_mapping_report, missing_values_report, sums_report

## Run the pipeline

In [28]:
budgets, missing_ids, column_mapping, missing_values, sums = do_pipeline()


 Loading files 

Loaded CP_2013.csv with encoding cp850


 Delete empty columns 


 Stripping padding from cells 

2013 stripped cell paddings

 Counting duplicate lines (NOT de-duplicating) 

2013 found 0 duplicate lines

 Mapping column to aliases 

2013 Descripción de Finalidad replaced with Descripción de Grupo Funcional
2013 Descripción de la entidad federativa replaced with Descripción de Entidad Federativa
2013 Finalidad replaced with Grupo Funcional

 Counting missing values 


 Building column mapping overview 

Column mapping overview: done

 Cleaning expenditure columns 

2013 cleaned and summed Aprobado = 4.32261895107e+12 pesos
2013 cleaned and summed Ejercicio = 4.6176176865e+12 pesos

 Breaking down Objeto del Gasto column 

Loaded catalog CAPITULO into "capitulo" (9 lines)
Loaded catalog CONCEPTO into "concepto" (63 lines)
Loaded catalog PARTIDA GENERICA into "partida_generica" (354 lines)
Loaded catalog PARTIDA ESPECÍFICA into "partida_específica" (668 lines)

2013 br

In [29]:
from gc import collect
collect()

2359

In [30]:
for year, budget in budgets.items():
    filepath = MERGED_FILE.replace('merged', str(year))
    budget.to_csv(filepath, encoding='utf-8', index=False)
    print('Saved', filepath)
    stdout.flush()

Saved pipeline.out/iteration-10-development/mexican_federal_budget.2013.csv


In [31]:
merged = concat(list(budgets.values()))
merged.to_csv(MERGED_FILE, encoding='utf-8', index=False)
print('Saved merged dataset to', MERGED_FILE)    

Saved merged dataset to pipeline.out/iteration-10-development/mexican_federal_budget.merged.csv


## Quality control

In [32]:
sorted(list(budget.columns))

['Actividad Institucional',
 'Aprobado',
 'Capitulo',
 'Ciclo',
 'Clave de cartera',
 'Concepto',
 'Descripción de Capitulo',
 'Descripción de Concepto',
 'Descripción de Entidad Federativa',
 'Descripción de Fuente de Financiamiento',
 'Descripción de Función',
 'Descripción de Grupo Funcional',
 'Descripción de Partida Específica',
 'Descripción de Partida Genérica',
 'Descripción de Programa Presupuestario',
 'Descripción de Ramo',
 'Descripción de Subfunción',
 'Descripción de Tipo de Gasto',
 'Descripción de Unidad Responsable',
 'Descripción de la Actividad Institucional',
 'Descripción de la modalidad del programa presupuestario',
 'Ejercicio',
 'Entidad Federativa',
 'Fuente de Financiamiento',
 'Función',
 'Grupo Funcional',
 'Modalidad del Programa presupuestario',
 'Partida Específica',
 'Partida Genérica',
 'Programa Presupuestario',
 'Ramo',
 'Subfunción',
 'Tipo de Gasto',
 'Unidad Responsable']

In [33]:
merged.sample(n=10)

,Ciclo,Ramo,Descripción de Ramo,Unidad Responsable,Descripción de Unidad Responsable,Grupo Funcional,Descripción de Grupo Funcional,Función,Descripción de Función,Subfunción,...,Aprobado,Ejercicio,Capitulo,Concepto,Descripción de Capitulo,Descripción de Concepto,Partida Genérica,Descripción de Partida Genérica,Partida Específica,Descripción de Partida Específica
21392,2013,7,Defensa Nacional,7.110,Dirección General de Administración,1,Gobierno,1.6,Seguridad Nacional,1.6.1,...,0.0,984514.40,2000,2100,Materiales y suministros,"Materiales de administracion, emision de docum...",212,Materiales y útiles de impresión y reproducción,21201,Materiales y útiles de impresión y reproducción
217562,2013,32,Tribunal Federal de Justicia Fiscal y Administ...,32.116,"Primera Sala Regional de Oriente, con sede en ...",1,Gobierno,1.2,Justicia,1.2.1,...,307188.0,338951.03,1000,1500,Servicios personales,Otras prestaciones sociales y económicas,154,Prestaciones contractuales,15403,Asignaciones adicionales al sueldo
301884,2013,GYN,Instituto de Seguridad y Servicios Sociales de...,GYN.GYN,Instituto de Seguridad y Servicios Sociales de...,2,Desarrollo Social,2.6,Protección Social,2.6.9,...,0.0,14778.00,1000,1500,Servicios personales,Otras prestaciones sociales y económicas,154,Prestaciones contractuales,15401,Prestaciones establecidas por condiciones gene...
6586,2013,4,Gobernación,4.K00,Instituto Nacional de Migración,1,Gobierno,1.3,Coordinación de la Política de Gobierno,1.3.7,...,9075.0,54496.80,3000,3500,Servicios generales,"Servicios de instalacion, reparacion, mantenim...",351,Conservación y mantenimiento menor de inmuebles,35101,Mantenimiento y conservación de inmuebles para...
176838,2013,21,Turismo,21.B00,Corporación Ángeles Verdes,3,Desarrollo Económico,3.7,Turismo,3.7.1,...,308533.0,253012.73,3000,3700,Servicios generales,Servicios de traslado y viáticos,371,Pasajes aéreos,37104,Pasajes aéreos nacionales para servidores públ...
49002,2013,9,Comunicaciones y Transportes,9.632,Centro SCT Guerrero,3,Desarrollo Económico,3.5,Transporte,3.5.1,...,1600.0,0.00,2000,2200,Materiales y suministros,Alimentos y utensilios,221,Productos alimenticios para personas,22104,Productos alimenticios para el personal en las...
83841,2013,9,Comunicaciones y Transportes,9.J4V,Fideicomiso de Formación y Capacitación para e...,3,Desarrollo Económico,3.5,Transporte,3.5.2,...,372060.0,363339.00,1000,1500,Servicios personales,Otras prestaciones sociales y económicas,159,Otras prestaciones sociales y económicas,15901,Otras prestaciones
155844,2013,16,Medio Ambiente y Recursos Naturales,16.RHQ,Comisión Nacional Forestal,3,Desarrollo Económico,3.2,"Agropecuaria, Silvicultura, Pesca y Caza",3.2.2,...,32967.0,17509.75,1000,1700,Servicios personales,Pago de estímulos a servidores públicos,171,Estímulos,17102,Estímulos al personal operativo
211059,2013,22,Instituto Federal Electoral,22.300,Juntas Distritales,1,Gobierno,1.3,Coordinación de la Política de Gobierno,1.3.6,...,195432.0,217013.07,1000,1400,Servicios personales,Seguridad social,144,Aportaciones para seguros,14404,Cuotas para el seguro de separación individual...
305136,2013,GYN,Instituto de Seguridad y Servicios Sociales de...,GYN.GYN,Instituto de Seguridad y Servicios Sociales de...,2,Desarrollo Social,2.6,Protección Social,2.6.9,...,0.0,68980.00,1000,1300,Servicios personales,Remuneraciones adicionales y especiales,133,Horas extraordinarias,13301,Remuneraciones por horas extraordinarias


In [34]:
objeto_breakdown = [
    'Ciclo', 
    'Capitulo', 'Concepto', 
    'Partida Específica', 
    'Partida Genérica'
]
merged[objeto_breakdown].sample(n=20)

,Ciclo,Capitulo,Concepto,Partida Específica,Partida Genérica
177532,2013,4000,4300,43801,438
75886,2013,2000,2900,29601,296
108410,2013,3000,3500,35901,359
223037,2013,2000,2200,22301,223
169014,2013,3000,3300,33604,336
90708,2013,2000,2400,24801,248
74380,2013,3000,3500,35501,355
204047,2013,3000,3100,31301,313
231719,2013,2000,2600,26107,261
267588,2013,1000,1500,15901,159


In [36]:
print('Total: missing', len(missing_ids), 'catalog IDs to breakdown the "Objeto del Gasto" column')
print('Tables:', dict(missing_ids.groupby('table').count()['ID']))
print('Years:', dict(missing_ids.groupby('year').count()['ID']))
try:
    missing_ids.sample(n=20)
except ValueError:
    pass

Total: missing 1 catalog IDs to breakdown the "Objeto del Gasto" column
Tables: {'partida_específica': 1}
Years: {2013: 1}


In [37]:
missing_ids.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 0 to 0
Data columns (total 3 columns):
ID       1 non-null object
table    1 non-null object
year     1 non-null int64
dtypes: int64(1), object(2)
memory usage: 32.0+ bytes


In [38]:
writer = ExcelWriter('test4.xlsx')    
no_dups = missing_ids.drop_duplicates('ID')
print(len(no_dups))
no_dups.to_excel(writer, 'sheet_name', encoding='utf-8', index=False)    

1


In [39]:
DataFrame([{'a': 1, "b": 2, 'c': 3}, {'a': 2, "b": 2, 'c': 4}]).drop_duplicates('b')

,a,b,c
0,1,2,3


In [40]:
column_mapping

,Column,2013
0,Ramo,True
1,Descripción de Función,True
2,Modalidad del Programa presupuestario,True
3,Tipo de Gasto,True
4,Ejercicio,True
5,Ciclo,True
6,Grupo Funcional,True
7,Entidad Federativa,True
8,Subfunción,True
9,Descripción de Unidad Responsable,True


In [41]:
missing_values

,Column,2013
0,Ramo,0
1,Descripción de Función,0
2,Modalidad del Programa presupuestario,0
3,Tipo de Gasto,0
4,Ejercicio,0
5,Ciclo,0
6,Grupo Funcional,0
7,Entidad Federativa,0
8,Subfunción,0
9,Descripción de Unidad Responsable,0


In [42]:
sums

,Column,2013
0,Ejercido,NaN
1,Devengado,NaN
2,Aprobado,4.322619e+12
3,Pagado,NaN
4,Modificado,NaN
5,Adefas,NaN
6,Ejercicio,4.617618e+12


In [43]:
merged.sample(n=20) 

,Ciclo,Ramo,Descripción de Ramo,Unidad Responsable,Descripción de Unidad Responsable,Grupo Funcional,Descripción de Grupo Funcional,Función,Descripción de Función,Subfunción,...,Aprobado,Ejercicio,Capitulo,Concepto,Descripción de Capitulo,Descripción de Concepto,Partida Genérica,Descripción de Partida Genérica,Partida Específica,Descripción de Partida Específica
287429,2013,GYN,Instituto de Seguridad y Servicios Sociales de...,GYN.GYN,Instituto de Seguridad y Servicios Sociales de...,2,Desarrollo Social,2.6,Protección Social,2.6.2,...,0.0,39317.00,1000,1500,Servicios personales,Otras prestaciones sociales y económicas,159,Otras prestaciones sociales y económicas,15901,Otras prestaciones
48862,2013,9,Comunicaciones y Transportes,9.632,Centro SCT Guerrero,3,Desarrollo Económico,3.5,Transporte,3.5.1,...,1500.0,0.00,2000,2100,Materiales y suministros,"Materiales de administracion, emision de docum...",211,"Materiales, útiles y equipos menores de oficina",21101,Materiales y útiles de oficina
131758,2013,15,"Desarrollo Agrario, Territorial y Urbano",15.132,Delegación Estatal en Guerrero,3,Desarrollo Económico,3.2,"Agropecuaria, Silvicultura, Pesca y Caza",3.2.1,...,54432.0,54432.00,1000,1500,Servicios personales,Otras prestaciones sociales y económicas,154,Prestaciones contractuales,15401,Prestaciones establecidas por condiciones gene...
303660,2013,GYN,Instituto de Seguridad y Servicios Sociales de...,GYN.GYN,Instituto de Seguridad y Servicios Sociales de...,2,Desarrollo Social,2.6,Protección Social,2.6.9,...,0.0,62435.00,1000,1200,Servicios personales,Remuneraciones al personal de carácter transit...,121,Honorarios asimilables a salarios,12101,Honorarios
157943,2013,16,Medio Ambiente y Recursos Naturales,16.RHQ,Comisión Nacional Forestal,3,Desarrollo Económico,3.2,"Agropecuaria, Silvicultura, Pesca y Caza",3.2.2,...,82236960.0,0.00,4000,4300,"Transferencias, asignaciones, subsidios y otra...",Subsidios y subvenciones,431,Subsidios a la producción,43101,Subsidios a la producción
46833,2013,9,Comunicaciones y Transportes,9.630,Centro SCT Durango,3,Desarrollo Económico,3.5,Transporte,3.5.1,...,8740.0,6117.99,3000,3500,Servicios generales,"Servicios de instalacion, reparacion, mantenim...",358,Servicios de limpieza y manejo de desechos,35801,"Servicios de lavandería, limpieza e higiene"
121776,2013,13,Marina,13.115,"Fuerzas, Regiones, Zonas y Sectores Navales",1,Gobierno,1.6,Seguridad Nacional,1.6.2,...,0.0,194.00,2000,2900,Materiales y suministros,"Herramientas, refacciones y accesorios menores",293,Refacciones y accesorios menores de mobiliario...,29301,Refacciones y accesorios menores de mobiliario...
37385,2013,9,Comunicaciones y Transportes,9.621,Centro SCT Aguascalientes,3,Desarrollo Económico,3.5,Transporte,3.5.4,...,30000.0,29849.61,2000,2100,Materiales y suministros,"Materiales de administracion, emision de docum...",216,Material de limpieza,21601,Material de limpieza
30638,2013,8,"Agricultura, Ganadería, Desarrollo Rural, Pesc...",8.310,Dirección General de Fomento a la Agricultura,3,Desarrollo Económico,3.2,"Agropecuaria, Silvicultura, Pesca y Caza",3.2.1,...,0.0,192700.00,4000,4300,"Transferencias, asignaciones, subsidios y otra...",Subsidios y subvenciones,431,Subsidios a la producción,43101,Subsidios a la producción
177787,2013,25,Previsiones y Aportaciones para los Sistemas d...,25.C00,Administración Federal de Servicios Educativos...,2,Desarrollo Social,2.5,Educación,2.5.1,...,1384284.0,684867.82,3000,3300,Servicios generales,"Servicios profesionales, cientificos, tecnicos...",339,"Servicios profesionales, científicos y técnico...",33901,Subcontratación de servicios con terceros


In [44]:
with open(join(ITERATION_FOLDER, BASENAME + '.missing.json')) as file:
    aliases = loads(file.read())
aliases['Descripción de Fuente de Financiamiento']

[]

In [45]:
breakdown = [
    'Ciclo', 
    'Capitulo', 
    'Concepto', 
    'Partida Genérica',        
    'Partida Específica', 
    'Descripción de Capitulo',
    'Descripción de Concepto', 
    'Descripción de Partida Genérica',
    'Descripción de Partida Específica'
]

merged[breakdown].sample(n=200)

,Ciclo,Capitulo,Concepto,Partida Genérica,Partida Específica,Descripción de Capitulo,Descripción de Concepto,Descripción de Partida Genérica,Descripción de Partida Específica
290249,2013,2000,2400,249,24901,Materiales y suministros,Materiales y articulos de construccion y de re...,Otros materiales y artículos de construcción y...,Otros materiales y artículos de construcción y...
267198,2013,1000,1300,133,13301,Servicios personales,Remuneraciones adicionales y especiales,Horas extraordinarias,Remuneraciones por horas extraordinarias
233262,2013,6000,6200,623,62302,Inversion publica,Obra publica en bienes propios,Construcción de obras para el abastecimiento d...,Mantenimiento y rehabilitación de obras para e...
191750,2013,3000,3900,399,39903,Servicios generales,Otros servicios generales,Otros servicios generales,Asignaciones a los grupos parlamentarios
282060,2013,2000,2200,223,22301,Materiales y suministros,Alimentos y utensilios,Utensilios para el servicio de alimentación,Utensilios para el servicio de alimentación
240940,2013,5000,5300,532,53201,"Bienes muebles, inmuebles e intangibles",Equipo e instrumental medico y de laboratorio,Instrumental médico y de laboratorio,Instrumental médico y de laboratorio
303047,2013,3000,3300,336,33605,Servicios generales,"Servicios profesionales, cientificos, tecnicos...","Servicios de apoyo administrativo, traducción,...",Información en medios masivos derivada de la o...
299317,2013,1000,1500,159,15901,Servicios personales,Otras prestaciones sociales y económicas,Otras prestaciones sociales y económicas,Otras prestaciones
55729,2013,3000,3500,351,35101,Servicios generales,"Servicios de instalacion, reparacion, mantenim...",Conservación y mantenimiento menor de inmuebles,Mantenimiento y conservación de inmuebles para...
158136,2013,4000,4300,433,43301,"Transferencias, asignaciones, subsidios y otra...",Subsidios y subvenciones,Subsidios a la inversión,Subsidios para inversión


In [46]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 308453 entries, 0 to 308452
Data columns (total 34 columns):
Ciclo                                                      308453 non-null object
Ramo                                                       308453 non-null object
Descripción de Ramo                                        308453 non-null object
Unidad Responsable                                         308453 non-null object
Descripción de Unidad Responsable                          308453 non-null object
Grupo Funcional                                            308453 non-null object
Descripción de Grupo Funcional                             308453 non-null object
Función                                                    308453 non-null object
Descripción de Función                                     308453 non-null object
Subfunción                                                 308453 non-null object
Descripción de Subfunción                                  308453 non